In [1]:
from data import LSHTCDataset
from embedding import Doc2Vec
from assemble_classifier import AssembleNoLabel, AssemblePredicted
%load_ext autoreload
%autoreload 2

In [2]:
data_name = "wiki_mediam"

In [3]:
dataset_train = LSHTCDataset(data_name, "train")
dataset_validate = LSHTCDataset(data_name, "validate")
dataset_test = LSHTCDataset(data_name, "test")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte

In [ ]:
for i in range(dataset_train.number_of_level()):
    print(dataset_train.check_each_number_of_class(i))

In [ ]:
import numpy as np
aa = []
dd = []
for i in range(11):
    start, end = dataset_test.level[i], dataset_test.level[i + 1]
    train = dataset_train.number_of_data_in_each_class()[start:end]
    validate = dataset_validate.number_of_data_in_each_class()[start:end]
    test = dataset_test.number_of_data_in_each_class()[start:end]
    sum_all = np.array(train) + np.array(validate) + np.array(test)
    zero_train = np.array(train) == 0
    zero_validate = np.array(validate) == 0
    zero_test = np.array(test) == 0
    zero_all = np.array(sum_all) <= 1
    zero_inverse_all = np.array(sum_all) > 1
    
#     print(i)
#     print(zero_train[zero_train & zero_validate].shape[0])
    print(zero_train[zero_all].shape[0])
    aa.append(zero_train[zero_all].shape[0])
    dd.append(zero_train[zero_inverse_all].shape[0])
#     print()
print(sum(aa))
print(sum(dd))
print(sum(aa) + sum(dd))


In [ ]:
import numpy as np
for i in range(11):
    start, end = dataset_test.level[i], dataset_test.level[i + 1]
    train = dataset_train.number_of_data_in_each_class()[start:end]
    test = dataset_test.number_of_data_in_each_class()[start:end]
    zero_train = np.array(train) < 20
    zero_test = np.array(test) < 1
    print(zero_train[zero_train].shape[0])

In [ ]:
doc2vec = Doc2Vec(dataset_train.number_of_classes(), size=150, epoch=250)
# doc2vec.fit(dataset_train.datas, dataset_train.labels, dataset_validate.datas, dataset_validate.labels)
doc2vec.load_model("best_now/doc2vec.model")

In [ ]:
dataset_train.change_to_Doc2Vec(doc2vec)
# dataset_validate.change_to_Doc2Vec(doc2vec)
# dataset_test.change_to_Doc2Vec(doc2vec)

In [ ]:
model = AssemblePredicted(data_name, dataset_train, dataset_validate, dataset_test, iteration=2000, batch_size=5000, hidden_size=[500,1500,2500,3500,3500,4000,4500,1500,1500,1500,1500,1500], target_hidden_size=[60,80,80,120,150,180,60,60,60,60,60], use_dropout=True, stopping_time=200, start_level=4, end_level=10000)
# model = AssembleNoLabel(data_name, dataset_train, dataset_validate, dataset_test, iteration=2000, batch_size=5000, hidden_size=[300,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500], use_dropout=False, start_level=2, end_level=2, stopping_time=200)

In [ ]:
model.train()

In [ ]:
model.tuning_threshold()

In [ ]:
f1_macro, f1_micro, f1_each = model.evaluate("train", correction=False)
print("F1 macro: %.4f F1 micro: %.4f" % (f1_macro, f1_micro))
for level, (macro, micro) in enumerate(f1_each):
    print("Level: %d F1 macro: %.4f F1 micro: %.4f" % (level, macro, micro))

In [ ]:
f1_macro, f1_micro, f1_each = model.evaluate("test", correction=False)
print("F1 macro: %.4f F1 micro: %.4f" % (f1_macro, f1_micro))
for level, (macro, micro) in enumerate(f1_each):
    print("Level: %d F1 macro: %.4f F1 micro: %.4f" % (level, macro, micro))